In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
latent_dim = 100

generator = keras.Sequential([
    keras.Input(shape=(latent_dim,)),
    layers.Dense(128 * 7 * 7),
    layers.Reshape((7, 7, 128)),
    layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
    layers.LeakyReLU(alpha=0.2),
    layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"),
    layers.LeakyReLU(alpha=0.2),
    layers.Conv2D(1, kernel_size=7, padding="same", activation="sigmoid"),
])
discriminator = keras.Sequential([
    keras.Input(shape=(28, 28, 1)),
    layers.Conv2D(128, kernel_size=3, strides=2, padding="same"),
    layers.LeakyReLU(alpha=0.2),
    layers.Conv2D(128, kernel_size=3, strides=2, padding="same"),
    layers.LeakyReLU(alpha=0.2),
    layers.Conv2D(128, kernel_size=3, strides=2, padding="same"),
    layers.LeakyReLU(alpha=0.2),
    layers.Flatten(),
    layers.Dropout(0.4),
    layers.Dense(1, activation="sigmoid"),
])
discriminator = keras.Sequential([
    keras.Input(shape=(28, 28, 1)),
    layers.Conv2D(128, kernel_size=3, strides=2, padding="same"),
    layers.LeakyReLU(alpha=0.2),
    layers.Conv2D(128, kernel_size=3, strides=2, padding="same"),
    layers.LeakyReLU(alpha=0.2),
    layers.Conv2D(128, kernel_size=3, strides=2, padding="same"),
    layers.LeakyReLU(alpha=0.2),
    layers.Flatten(),
    layers.Dropout(0.4),
    layers.Dense(1, activation="sigmoid"),
])
discriminator.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5), loss="binary_crossentropy")
discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.Model(gan_input, gan_output)
gan.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5), loss="binary_crossentropy")
(x_train, _), (_, _) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
epochs = 30
batch_size = 128

for epoch in range(epochs):
    for batch in range(x_train.shape[0] // batch_size):
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        generated_images = generator.predict(noise)
        real_images = x_train[np.random.randint(0, x_train.shape[0], batch_size)]
        combined_images = np.concatenate([real_images, generated_images])
        labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
        labels += 0.05 * np.random.random(labels.shape)
        discriminator_loss = discriminator.train_on_batch(combined_images, labels)

        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        misleading_targets = np.zeros((batch_size, 1))
        gan_loss = gan.train_on_batch(noise, misleading_targets)

    print(f"Epoch {epoch+1}, Discriminator Loss: {discriminator_loss}, GAN Loss: {gan_loss}")

# Generate and plot some images
noise = np.random.normal(0, 1, (10, latent_dim))
generated_images = generator.predict(noise)

for i in range(10):
    plt.imshow(generated_images[i].reshape(28, 28), cmap='gray')
    plt.show()


4/4 [==============================] - 1s 154ms/step


KeyboardInterrupt: 